# Empirical Project 4


## Getting Started in Python

## Preliminary Settings

Let's import the packages we'll need and also configure the settings we want:

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pingouin as pg
from skimpy import skim
from lets_plot import *

LetsPlot.setup_html(no_js=True)


- Go to the United Nations’ [National Accounts Main Aggregates Database website](https://tinyco.re/7226184). On the right-hand side of the page, under ‘Data Availability’, click ‘Downloads’.
- Under the subheading ‘GDP and its breakdown at constant 2015 prices in US Dollars’, select the Excel file ‘All countries for all years – sorted alphabetically’.
- Save it in a subfolder of the directory you are coding in such that that the relative path is `data/Download-GDPconstant-USD-all.xlsx`.

## Python Walkthrough 4.1

**Importing the Excel file (`.xlsx` or `.xls`) into Python**

First, make sure you move the saved the data to a folder called `data` that is a subfolder of your working directory. The working directory is the folder that your code 'starts' in, and the one that you open when you start Visual Studio Code. Let's say you called it `core`, then the file and folder structure of your working directory would look like this:

```bash
📁 core
│──📁data
   └──Download-GDPconstant-USD-all.xlsx
│──empirical_project_4.py
```

This is similar to what you should see in Visual Studio Code under the explorer tab (although the working directory, `core`, won't appear). You can check your working directory by running

```python
import os
os.getcwd()
```

in Visual Studio Code.

Before importing the file into Python, open the file in Excel, OpenOffice, LibreOffice, or Numbers to see how the data is organized in the spreadsheet, and note that:

- There is a heading that we don’t need, followed by a blank row.
- The data we need starts on row three.

Armed with this knowledge, we can import the data using the `Path` module to create the path to the data:

In [ ]:
df = pd.read_excel(Path("data/Download-GDPconstant-USD-all.xlsx"), skiprows=2)
df.head()

## Python Walkthrough 4.2

**Making a frequency table**

We want to create a table showing how many years of `Final consumption expenditure` data are available for each country.

Looking at the dataset’s current format, you can see that countries and indicators (for example, `Afghanistan` and `Final consumption expenditure`) are row variables, while year is the column variable. This data is organized in ‘wide’ format (each individual’s information is in a single row).

For many data operations and making charts it is more convenient to have indicators as column variables, so we would like `Final consumption expenditure` to be a column variable, and year to be the row variable. Each observation would represent the value of an indicator for a particular country and year. This data is organized in ‘long’ format (each individual’s information is in multiple rows). This is also called 'tidy' data and it can be recognised by having variable per column and one observation per row. Many data scientists consider keeping data in tidy format good practice.

To change data from wide to long format, we use the `pd.melt` method. The `melt` method is very powerful and useful, as you will find many large datasets are in wide format. In this case, `pd.melt` takes the data from all columns not specified as being `id_vars` (via a list of column names), and uses them to create two new columns: one contains the name of the row variable created from the former column names, which is the year here; we can set that new column's name with `var_name="year"`. The second new column contains the values that were in the columns we unpivoted and is automatically given the name `value`. (We could have set a new name for this column by passing `value_name=` too.)

Compare `df_long` to the wider `df` to understand how the melt command works. To learn more about organizing data in Python, see the [Working with Data](https://aeturrell.github.io/coding-for-economists/data-intro.html) section of 'Coding for Economists'. 

In [ ]:
df_long = pd.melt(
    df, id_vars=["Area/CountryID", "Area/Country", "IndicatorName"], var_name="year"
)
df_long.head()

To create the required table, we only need `Final consumption expenditure` of each country, which we extract using the `.loc` function. We'd like all columns so we pass the condition in the first position of `.loc` and leave the second entry as `:` for all columns.

In [ ]:
cons = df_long.loc[df_long["IndicatorName"] == "Final consumption expenditure", :]

Now let's create our table. 

In [ ]:
year_count = cons.groupby("Area/Country").agg(available_years=("year", "count"))
year_count

Translating the code in words: Take the variable `cons` and group the observations by area and country (`.groupby(Area/Country")`), then take this result and aggregate `.agg` it such that a new variable called available years (`available_years=`) is created that sees the column year counted (`("year", "count")`).

Now we can establish how many of the 250 countries and areas in the dataset have complete information. A dataset is complete if it has the maximum number of available observations (given by `year_count["available_years"].max()`).

In [ ]:
sum(year_count["available_years"] == year_count["available_years"].max())

In this case, the full set of data are available for all countries and areas.

## Python Walkthrough 4.3

**Creating new variables**

We will use Brazil, the US, and China as examples.

Before we select these three countries, we will calculate the net exports (exports minus imports) for all countries, as we need that information in Python walkthrough 4.4. We will also shorten the names of the variables we need, to make the code easier to read. We will use a dictionary to map names into shorter formats. A dictionary is a built-in object type in Python and always has the structure `{key1: value1, key2: value2, ...}` where the keys and values could have any type (eg string, int, dataframe). In our case, both keys and values will be strings. We will use a convention for our naming that is known as "snake case". This means all lower case with spaces replaced by underscores (it looks a bit like a snake!). There are packages that will auto-rename long variables for you, but let's see how to do it manually here.

In [ ]:
short_names_dict = {
    "Final consumption expenditure": "final_expenditure",
    "Household consumption expenditure (including Non-profit institutions serving households)": "hh_expenditure",
    "General government final consumption expenditure": "gov_expenditure",
    "Gross capital formation": "capital",
    "Imports of goods and services": "imports",
    "Exports of goods and services": "exports",
}
# rename these values
df_long["IndicatorName"] = df_long["IndicatorName"].replace(short_names_dict)

`df_long` still has several rows for a particular country and year (one for each indicator). We will reshape this data using the `.pivot` method to ensure that we have only one row per country/area and per year. Note that `pivot` preserves the list of columns we pass as the `index=` and pivots the columns we pass to `columns=` out so that they are wide.

In [ ]:
df_table = df_long.pivot(
    index=["Area/CountryID", "Area/Country", "year"], columns=["IndicatorName"]
)
df_table.head()

Note that `df_long` previously had information in 5 columns, "Area/CountryID", "Area/Country", and "year", which are still columns as we declared them as the index of the new dataframe; "IndicatorName", the content of which has now become the variable names; and data in the "values" column. That last column was not mentioned in the previous line of code (it was clear to **pandas** what had been left out) but it is the values of that column that now appear in the body of the `df_table` dataframe. See what happens if you leave two of the original columns unmentioned, for instance take "year" out of the index list. You should find that you get an error message because it is then ambigious as to how to create `df_table` from `df_long`.


Now we create a `net_exports` column based on the existing columns (exports - imports), and we can know that this will be a unique country/area and year combination for each row. First we need to drop the top level of the column index, which is currently called `value`: we don't need this anymore. This will allow for direct access to the `exports` and `imports` columns. We'll also reset the index to row numbers rather than those three columns we used in the pivot. We'll also remove the name of the columns as we won't need that any longer.

In [ ]:
df_table.columns = df_table.columns.droplevel()
df_table = df_table.reset_index()
df_table.columns.name = ""
df_table["net_exports"] = df_table["exports"] - df_table["imports"]

Let us select our three chosen countries to check that we calculated net exports correctly.

In [ ]:
sel_countries = ["Brazil", "United States", "China"]
cols_to_keep = ["Area/Country", "year", "exports", "imports", "net_exports"]

df_sel_un = df_table.loc[df_table["Area/Country"].isin(sel_countries), cols_to_keep]
df_sel_un.head()

## Python Walkthrough 4.4

**Plotting and annotating time series data**

*Extract the relevant data*

We will work with the `df_long` dataset, as the long format is well suited to produce charts with the **lets-plot** package. In this example, we use the US and China (which we will now save as the dataframe `comp`).

In [ ]:
# take a copy of df_long that is just US and China
comp = df_long.loc[df_long["Area/Country"].isin(["United States", "China"]), :].copy()
# Convert value to billion USD
comp["value"] = comp["value"] / 1.0e9
# Filter down to certain cols and values
comp = comp.loc[
    comp["IndicatorName"].isin(
        ["gov_expenditure", "hh_expenditure", "capital", "imports", "exports"]
    ),
    ["Area/Country", "year", "IndicatorName", "value"],
]
comp.head()

*Plot a line chart*

We can now plot this data using the **lets-plot** data visualisation library. We'll subset to US data.

In [ ]:
(
    ggplot(
        comp.loc[comp["Area/Country"] == "United States", :],
        aes(
            x="year",
            y="value",
            color="IndicatorName",
            linetype="IndicatorName",
        ),
    )
    + geom_line(size=1)
)

*Figure 4.2 The US’s GDP components (expenditure approach).*

There are plenty of problems with this chart:

- the vertical axis label is uninformative
- there is no chart title
- the y-axis dips below zero
- the legend is uninformative.


To improve this chart, we add features to the figure by creating the axis, `ax`, explicitly. We'll also use a trick where we invert the dictionary from earlier and use this to supply full names to the legend via a new "indicator" column.

Earlier, we had a dictionary, `short_names_dict`, that mapped the long names, eg "Final consumption expenditure", into short names, eg "final_expenditure". For making our plot, we'd like to reverse this. The first line of code helps with this by reversing the earlier dictionary—you don't need to understand the details of this, just know that it's an inversion so that "final_expenditure" maps into "Final consumption expenditure" and so on.

In [ ]:
# reverse the dictionary from earlier
rev_name_dict = {v: k.split("(")[0] for k, v in short_names_dict.items()}
# create a new col with the original names
comp["Indicator"] = comp["IndicatorName"].replace(rev_name_dict)

(
    ggplot(
        comp.loc[comp["Area/Country"] == "United States", :],
        aes(x="year", y="value", color="Indicator", linetype="Indicator"),
    )
    + geom_line(size=2)
    + labs(
        y="Billions USD",
        title="GDP components over time",
        color="Indicator",
    )
    + ggsize(800, 500)
    + scale_x_continuous(format="")
)

***Figure 4.3** The US’s GDP components (expenditure approach), amended chart.*

We can make a chart for more than one country simultaneously using `comp` rather than just a slice of `comp`.

In [ ]:
(
    ggplot(comp, aes(x="year", y="value", color="Indicator", linetype="Indicator"))
    + geom_line(size=2)
    + labs(
        y="Billions USD",
        title="GDP components over time",
        color="Indicator",
    )
    + ggsize(1000, 500)
    + scale_x_continuous(format="")
    + facet_wrap(facets="Area/Country", ncol=2)
)

***Figure 4.4** GDP components over time (China and the US).*

## Python Walkthrough 4.5

**Calculating new variables and plotting time series data**

*Calculate proportion of total GDP*

We will use the `comp` dataset created in Python Walkthrough 4.4. First we will calculate net exports, as that contributes to GDP. As the data is currently in long format, we will reshape the data into wide format so that the variables we need are in separate columns instead of separate rows (using the `pivot` method, as in Python Walkthrough 4.3), calculate net exports, then transform the data back into long format using the `melt` method.

On the way, we'll end up dropping the Indicator Names, and dropping the top level "value"

In [ ]:
comp_wide = comp.drop("Indicator", axis=1).pivot(
    index=["Area/Country", "year"], columns="IndicatorName"
)
comp_wide.columns = comp_wide.columns.droplevel()
comp_wide = comp_wide.reset_index()
comp_wide.head()

In order to figure out what `.droplevel()` and `.reset_index()` do, you should comment out each of the two lines in turn and re-run the block of code to see what difference these lines make.

Add the new column for net exports = exports – imports

In [ ]:
comp_wide["net_exports"] = comp_wide["exports"] - comp_wide["imports"]
comp_wide.head()

Return to long format with the household expenditure, capital, and net export variables

In [ ]:
comp2 = pd.melt(
    comp_wide.drop(["exports", "imports"], axis=1),
    id_vars=["year", "Area/Country"],
    var_name="indicator",
    value_name="2015_bn_usd",
)
comp2.head()

Now we will create a new dataframe (`props`) also containing the proportions for each GDP component (`proportion`), using method chaining to link functions together.

In [ ]:
props = comp2.assign(
    proportion=comp2.groupby(["Area/Country", "year"])["2015_bn_usd"].transform(
        lambda x: x / x.sum()
    )
)

In words, we did the following: Take the `comp2` dataframe and add in a new column called `proportion` (this bit starts with `.assign(proportion=`) that, within area and year groups (`.groupby(["Area/Country", "year"])`) takes the value (`["2015_bn_usd"]`) and divides it by the total value for that group (`.transform(lambda x: x/x.sum())`). For example, the first row gives the proportion of capital for China in 1970.

The result is then saved in props. Look at the props dataframe to confirm that the above command has achieved the desired result. (You can check the answer with `props.groupby(["Area/Country", "year"])["proportion"].sum()`.)

*Plot a line chart*

Now we redo the line chart from Python Walkthrough 4.4 using the variable `props`.

In [ ]:
# Update dictionary for net exports, which is new
rev_name_dict.update({"net_exports": "Net exports"})
props["Component"] = props["indicator"].map(rev_name_dict)
(
    ggplot(
        props, aes(x="year", y="proportion", color="Component", linetype="Component")
    )
    + geom_line(size=2)
    + facet_wrap(facets="Area/Country", ncol=2)
)

***Figure 4.5** GDP component proportions over time (China and the US).*

## Python Walkthrough 4.6

**Creating stacked bar charts**

*Calculate proportion of total GDP*

This walk-through uses the following countries (chosen at random):

- developed countries: Germany, Japan, United States
- transition countries: Albania, Russian Federation, Ukraine
- developing countries: Brazil, China, India.

The relevant data are still in the `df` dataframe. Before we select these countries, we first calculate the required proportions for all countries for capital, final expenditure, and net exports (out of those columns).

In [ ]:
columns_to_track = ["capital", "final_expenditure", "net_exports"]
countries_to_use = [
    "Germany",
    "Japan",
    "United States",
    "Albania",
    "Russian Federation",
    "Ukraine",
    "Brazil",
    "China",
    "India",
]

# Find the proportions for these columns and create new columns called "prop_" + original col name
for col in columns_to_track:
    df_table["prop_" + col] = df_table[col].divide(
        df_table[columns_to_track].sum(axis=1)
    )

# filter this down to 2015 for the countries and cols we want
cols_to_keep = ["prop_" + col for col in columns_to_track] + ["Area/Country", "year"]
df_2015 = df_table.loc[
    (df_table["Area/Country"].isin(countries_to_use)) & (df_table["year"] == 2015),
    cols_to_keep,
]
df_2015.head()

*Plot a stacked bar chart*

Now let’s create the bar chart. First we need to melt the data into a format where each row is an observation, each column a variable.

In [ ]:
df_2015_melt = pd.melt(
    df_2015,
    id_vars=["Area/Country", "year"],
    value_name="Percent",
    var_name=["Component"],
)
df_2015_melt["Percent"] = df_2015_melt["Percent"] * 100

In [ ]:
(
    ggplot(df_2015_melt, aes(x="Area/Country", y="Percent", fill="Component"))
    + geom_bar(stat="identity")
    + coord_flip()
    + labs(fill="Components of GDP")
)

***Figure 4.6** GDP component proportions in 2015.*

Note that even when a country has a trade deficit (proportion of net exports < 0), the proportions will add up to 1, but the proportions of final expenditure and capital will add up to more than 1.

**FIND OUT MORE**

*The natural log: What it means, and how to calculate it in Python*

The natural log turns a linear variable into a concave variable, as shown in Figure 4.9. For any value of income on the horizontal axis, the natural log of that value on the vertical axis is smaller. At first, the difference between income and log income is not that big (for example, an income of 2 corresponds to a natural log of 0.7), but the difference becomes bigger as we move rightwards along the horizontal axis (for example, when income is 100,000, the natural log is only 11.5).

![](https://www.core-econ.org/doing-economics/book/images/web/figure-04-05.jpg)

***Figure 4.9** Comparing income with the natural logarithm of income.*

The reason why natural logs are useful in economics is because they can represent variables that have diminishing marginal returns: an additional unit of input results in a smaller increase in the total output than did the previous unit. (If you have studied production functions, then the shape of the natural log function might look familiar.)

When applied to the concept of wellbeing, the ‘input’ is income, and the ‘output’ is material wellbeing. It makes intuitive sense that a $100 increase in per capita income will have a much greater effect on wellbeing for a poor country compared to a rich country. Using the natural log of income incorporates this notion into the index we create. Conversely, the notion of diminishing marginal returns (the larger the value of the input, the smaller the contribution of an additional unit of input) is not captured by GDP per capita, which uses actual income and not its natural log. Doing so makes the assumption that a $100 increase in per capita income has the same effect on wellbeing for rich and poor countries.

The **numpy** log function in Python calculates the natural log of a value for you. To calculate the natural log of a value, `x`, type `np.log(x)`. If you have a scientific calculator, you can check that the calculation is correct by using the ln or log key.

Now that you know about the natural log, you might want to go back to Question 3(c) in Part 4.1, and create a new chart using the natural log scale. The natural log is used in economics because it approximates percentage changes i.e. log(x) – log(y) is a close approximation to the percentage change between x and y. So, using the natural log scale, you will be able to ‘read off’ the relative growth rates from the slopes of the different series you have plotted. For example, a 0.01 change in the vertical axis value corresponds to a 1% change in that variable. This will allow you to compare the growth rates of the different components of GDP.

## Python Walkthrough 4.7

**Calculating the HDI**

We will use `pd.read_excel` to import the data file, which we saved using its default name of '2020_Statistical_Annex_Table_1.xlsx’ in the data folder within our working directory. Before importing, look at the Excel file so that you understand its structure and how it corresponds to the code options used below. It's a long way from being a neat and tidy dataset! We will save the imported data as the `df_hdr` dataframe. Having taken a look at the Excel file, we can see we should skip the first few rows and take row 1 as the header (which becomes our column names).



In [ ]:
df_hdr = pd.read_excel(
    Path("data/2020_Statistical_Annex_Table_1.xlsx"), skiprows=3, header=1
)
df_hdr.head()

Looking at the `df_hdr` dataframe, there are rows that have information that isn’t data (for example, all the rows with an ‘NaN’ in), as well as variables/columns that do not contain data, or are a mix of 'NaN' and data.

Cleaning up the dataframe can be easier to do in Excel by deleting irrelevant rows and columns, but one advantage of doing it in Python is replicability. Suppose in a year’s time you carried out the analysis again with an updated spreadsheet containing new information. If you had done the cleaning in Excel, you would have to redo it from scratch, but if you had done it in Python, you can simply rerun the code below. (This works for new data that are in the same format too.)

Let's do some data cleaning.

First, we rename the last column by picking up the year entry below it in order to distinguish between different years of HDI rank. Next, we replace any columns that are "Unnamed" with entries from the first row of observations with a list comprehension. Then we eliminate rows that do not have any numbers in the `"HDI rank"` column.

In [ ]:
df_hdr = df_hdr.rename(columns={"HDI rank": "HDI_rank_" + str(df_hdr.iloc[1, -1])})
df_hdr.columns = [
    df_hdr.columns[i] if "Unnamed" not in df_hdr.columns[i] else df_hdr.iloc[0, i]
    for i in range(len(df_hdr.columns))
]
df_hdr = df_hdr.loc[~pd.isna(df_hdr["HDI rank"]), :]
df_hdr.head()

Now we can eliminate rows in HDI rank that do not have numbers in, and, following that, eliminate columns that contain NaNs.

In [ ]:
df_hdr = df_hdr.loc[~pd.isna(df_hdr["HDI_rank_2018"]), :]
df_hdr = df_hdr.dropna(axis=1, how="any")
df_hdr.head()

Now let's switch to shorter column names and check what datatypes we have in our data:

In [ ]:
new_column_names = [
    "hdi_rank",
    "country",
    "hdi",
    "life_exp",
    "exp_yrs_school",
    "mean_yrs_school",
    "gni_capita",
    "gni_hdi_rank",
    "hdi_rank_2018",
]
df_hdr.columns = new_column_names
df_hdr.info()

Looking at the structure of the data, we see that **pandas** thinks that all the data are objects because the original datafile contained non-numerical entries (these rows have now been deleted). Apart from the `"country"` variable, which we want to be a categorical variable, all variables should be doubles or ints. 

Now, we could do this individually, by running:

```python
df_hdr = df_hdr.astype({"hdi_rank": "int32"})
```

and similar for each name and its corresponding type. Note that the structure `{ ... : ...}` is a dictionary, which has *keys* and *values*.

However, it would be quite tedious to have to retype the same line over and over again. Instead we can pass on dictionary with key-value pairs for all mappings of names to data types we'd like to have in our dataframe.

To do this, we'll use a trick where we 'zip' two variables (the column names and datatypes) together into a dictionary that maps the column name into the datatype we'd it to have. The zip function brings an element of two lists together in turn, a bit like a zipper on your clothes brings two interlocking plastic teeth together.

In [ ]:
new_column_datatypes = [
    "int",
    "category",
    "double",
    "double",
    "double",
    "double",
    "double",
    "int",
    "int",
]
dict_of_names_to_types = {k: v for k, v in zip(new_column_names, new_column_datatypes)}
dict_of_names_to_types

We now have a dictionary of all names and their associated types that we can use:

In [ ]:
df_hdr = df_hdr.astype(dict_of_names_to_types)
df_hdr.info()

Now we have a nice clean dataset that we can work with.

We start by calculating the three indices, using the information given. For the education index we calculate the index for expected and mean schooling separately, then take the arithmetic mean to get `i_education`. As some mean schooling observations exceed the specified ‘maximum’ value of 18, the calculated index values would be larger than 1. To avoid this, we first replace these observations with 18 to obtain an index value of 1.

In [ ]:
df_hdr.loc[df_hdr["exp_yrs_school"] > 18, "exp_yrs_school"] = 18

# Now create the indices
df_hdr["i_health"] = (df_hdr["life_exp"] - 20) / (85 - 20)
df_hdr["i_education"] = (
    ((df_hdr["exp_yrs_school"] - 0) / (18 - 0))
    + (df_hdr["mean_yrs_school"] - 0) / (15 - 0)
) / 2
df_hdr["i_income"] = (np.log(df_hdr["gni_capita"]) - np.log(100)) / (
    np.log(75000) - np.log(100)
)
df_hdr["hdi_calc"] = np.power(
    df_hdr["i_health"] * df_hdr["i_education"] * df_hdr["i_income"], 1 / 3
)

Now we can compare the `HDI` given in the table and our calculated HDI.

In [ ]:
df_hdr[["hdi", "hdi_calc"]]

The HDI is one way to measure wellbeing, but you may think that it does not use the most appropriate measures for the non-material aspects of wellbeing (health and education).

https://databank.worldbank.org/

Now we will use the same method to create our own index of non-material wellbeing (an ‘alternative HDI’), using different indicators. You can find alternative indicators to measure health and education on the [World Bank data bank](https://databank.worldbank.org) under "World Development Indicators". Use the interactive menus to select all years, all countries, and then the indicators from the Walkthrough below.

1. Create an alternative index of wellbeing. In particular, propose alternative Education and Health indices in (a) and (b), then combine these with the existing Income index in (c) to calculate an alternative HDI. Examine whether the changes caused substantial changes in country rankings in (d).

a) Choose two to three indicators to measure health, and two to three indicators to measure education. Explain why you have chosen these indicators.

b) Carefully merge the data into your existing data. Choose a reasonable maximum and minimum value for each indicator and justify your choices.

c) Calculate your alternative versions of the education and health dimension indices. Since you have chosen more than one indicator for this dimension, make sure to average the dimension indices as done in Question 3
(b). Also ensure that higher indicator values always represent better outcomes. Now calculate the alternative HDI as done in Questions 3 and 4. Remember to combine your alternative education and health indices with the existing income index from Question 2.

d) Create a new variable showing each country’s rank according to your alternative HDI, where 1 is assigned to the country with the highest value. Compare your ranking to the HDI rank. Are the rankings generally similar, or very different? (See R walk-through 4.8 on how to do this.)

## Python Walkthrough 4.8

**Creating your own HDI**

*Merge data and calculate alternative indices*

This example uses the following indicators:

- Education: Literacy rate, adult (% ages 15 and older); School enrolment, tertiary (% gross); Trained teachers in primary education (%)

- Health: Prevalence of stunting, height for age (% under age 5); Mortality rate, female adult (per 1,000 female adults); Mortality rate, male adult (per 1,000 male adults).

- Income: GNP per capita (in constant 2015 USD).

You'll need to select these indices and then use the 'Download Options' buttons to get them as a CSV file.

First, we import the data and check that it has been imported correctly. You can see that each row represents a different country, and each column represents a different year-indicator combination. Note that the data come with a good number of empty rows at the end before some meta-data that have been inserted, so we drop those rows that have NaNs in the Country Code column.

In [ ]:
all_hdr = pd.read_csv(Path("data/world_bank_data.csv")).dropna(
    subset=["Country Code"], axis=0
)
all_hdr

Then we need to filter to just the year we want. We'll do this in a new data frame. Note that two dots (`..`) are a string that indicates a missing variable. Unfortunately, if we have a mix of strings and other data types (like double), **pandas** isn't sure how to treat the column. So we need to replace the code for missing values in a string, "..", with "proper" numeric missing values which will allow us to do operations like arithmetic on the column. To do this, we use the `pd.to_numeric` function—once it has been applied using the `errors="coerce"` option, you'll see that any occurrences of `..` have been converted to `NaN`, which means not a number.

In [ ]:
hdr_2018 = all_hdr.loc[
    :, ["Country Name", "Country Code", "Series Name", "Series Code", "2018 [YR2018]"]
]
hdr_2018["2018 [YR2018]"] = pd.to_numeric(
    hdr_2018["2018 [YR2018]"], errors="coerce"
).fillna(np.nan)
hdr_2018.sample(5)

For ease, we'll introduce some short name versions of the series names and we will do some additional data typing

In [ ]:
dict_to_shorter_hdi_names = {
    "SE.TER.ENRR": "enrollment",
    "SE.PRM.TCAQ.ZS": "trained_teachers",
    "SE.ADT.LITR.ZS": "literacy",
    "SP.DYN.AMRT.MA": "mortality_m",
    "SP.DYN.AMRT.FE": "mortality_f",
    "SH.STA.STNT.ZS": "stunting",
    "NY.GNP.PCAP.KD": "income",
}

dict_of_types = {
    "Country Name": "category",
    "Country Code": "category",
    "Series Code": "category",
    "indicator": "category",
    "Series Name": "string",
}

hdr_2018["indicator"] = hdr_2018["Series Code"].map(dict_to_shorter_hdi_names)
hdr_2018 = hdr_2018.astype(dict_of_types)
hdr_2018.head()

Now we can look at some summary statistics; we'll use the **skimpy** package.

You will need to install skimpy on the command line of your computer by typing

```bash
pip install skimpy
```

before running the code below.

In [ ]:
skim(hdr_2018)

Before we can calculate indices, we need to set minimum and maximum values, which we base on the minimum and maximum values in the sample. Let's look at the max and min by indicator:

In [ ]:
min_max = hdr_2018.groupby(["indicator"])["2018 [YR2018]"].agg(["min", "max"])
min_max

We want observations to be within the interval of min to max. Let's do this systematically for all of the indicators:

In [ ]:
hdr_2018["index_indicator"] = hdr_2018.groupby(["indicator"])[
    "2018 [YR2018]"
].transform(lambda x: (x - x.min()) / (x.max() - x.min()))
hdr_2018.head(7)

You should be able to see that the first row is the original value for literacy for Sri Lanka with the minimum literacy subtracted divided by the difference between the max and min for literacy.

Now we'll build the two new indicators based on subsets of the existing indicators. To do this, we'll aggregate over existing indicators—so we're going to end up a smaller data frame that only has each country in it once. For neatness, we can also pack these into a single dataframe.

In [ ]:
# create educ index
educ_index_indicators = ["literacy", "enrollment", "trained_teachers"]
hdr_2018_educ = (
    hdr_2018.loc[hdr_2018["indicator"].isin(educ_index_indicators), :]
    .groupby(["Country Code"])["index_indicator"]
    .mean()
)

# create health index. These indicators are less good when the number is high
# so we invert this measure
health_index_indicators = ["stunting", "mortality_m", "mortality_f"]
hdr_2018_health = (
    1.0
    - hdr_2018.loc[hdr_2018["indicator"].isin(health_index_indicators), :]
    .groupby(["Country Code"])["index_indicator"]
    .mean()
)

# create income indicator
hdr_2018_income = (
    hdr_2018.loc[hdr_2018["indicator"].isin(["income"]), :]
    .groupby(["Country Code"])["index_indicator"]
    .mean()
)

# combine them all
hdr_2018_long = pd.concat([hdr_2018_educ, hdr_2018_health, hdr_2018_income], axis=1)
# give each sensible names (they both inherit column name of "index_indicator" by default)
hdr_2018_long.columns = ["educ_index", "health_index", "income_index"]
skim(hdr_2018_long.reset_index())

We'd like to merge our two new indicators back into the main dataset on the same basis as the existing ones. Our new indices are in a different shape, so to do this we have to either reshape `hdr_2018` to be in wide format or reshape `hdr_2018_long` to be in wide format. Let's try the latter, adding in the "missing" columns as we go

In [ ]:
new_indices_wide = pd.melt(
    hdr_2018_long.reset_index(),
    id_vars="Country Code",
    var_name="indicator",
    value_name="index_indicator",
)
country_code_to_name = dict(
    zip(
        hdr_2018["Country Code"].drop_duplicates(),
        hdr_2018["Country Name"].drop_duplicates(),
    )
)
new_indices_wide["Series Name"] = new_indices_wide["indicator"]
new_indices_wide["Series Code"] = new_indices_wide["indicator"]
new_indices_wide["Country Name"] = new_indices_wide["Country Code"].map(
    country_code_to_name
)
hdr_2018 = pd.concat([new_indices_wide, hdr_2018], axis=0)
hdr_2018

Now we have added both of the alternative indicators in, we are now in a position to compute our own HDI indicator. First, we'll flip the data to wide format

In [ ]:
hdr_2018_w = hdr_2018.pivot(
    index=["Country Code", "Country Name"],
    columns="indicator",
    values="index_indicator",
)
hdr_2018_w = hdr_2018_w.assign(
    hdi_own=lambda x: np.power(
        x["educ_index"] * x["health_index"] * x["income_index"], 1 / 3
    )
)
skim(hdr_2018_w)

We can see that our shiny new HDI has 80 missing values (37% of all entries) because of the high number of entries that feed into it being invalid too.

### Calculate Ranks

To compare the ranks of the two indices (the original HDI and our alternative HDI), we should only rank the countries that have observations for both indices. We will create a dataframe called `HDR2018_sub` that contains this subset of countries.

In [ ]:
# drop columns we're not now interested in
hdr_2018_w_sub = hdr_2018_w.dropna(subset=["hdi_own"]).drop(
    health_index_indicators + educ_index_indicators + ["income"], axis=1
)
hdr_2018_w_sub

It looks like we retain 137 countries for which there is sufficient information to produce our special HDI.

We'll now attempt to merge `hdr_2018_w_sub`, which has our hdi indicator in it, with the one we prepared earlier. The only overlapping column is one that contains country name—though this is called something different in each dataset, so we must rename the relevant column in at least one of the two dataframes when merging.

In [ ]:
hdr_merged = pd.merge(
    hdr_2018_w_sub.reset_index().rename(columns={"Country Name": "country"}),
    df_hdr.drop(
        ["mean_yrs_school", "life_exp", "exp_yrs_school", "gni_capita"], axis=1
    ),
    on=["country"],
    how="inner",
)
hdr_merged

So we have a succesful inner merge for 120 countries. Finally, we're going to compute a rank of countries based on our own hdi in 2018. We use the `ascending=False` option in the `.rank` method so as to make the "best" value have rank 1.

In [ ]:
hdr_merged["hdi_own_rank"] = hdr_merged["hdi_own"].rank(ascending=False).astype("int")

With this done, we can make a scatter plot of the ranks compared with one another

In [ ]:
(ggplot(hdr_merged, aes(x="hdi_rank_2018", y="hdi_own_rank")) + geom_point(size=3))

**Figure 4.10 Scatterplot of ranks for HDI and alternative HDI index.**

We can improve this chart quite a bit. One particularly useful addition would be some text showing some key values. Using **lets-plots**' system for mapping variables onto layers of a chart, we can do this fairly easily using long format data by adding a column called "text" that has an entry only every ten values (otherwise it would overwhelm the chart). We first sort by the 2018 official HDI rank, then create an empty text column, and then add the real country name into the text column only for every tenth entry.

In [ ]:
hdr_merged = hdr_merged.sort_values("hdi_rank_2018")
hdr_merged["text"] = ""
hdr_merged.iloc[::10, -1] = hdr_merged.iloc[::10, 1]
hdr_merged.head()

In [ ]:
(
    ggplot(hdr_merged, aes(x="hdi_rank_2018", y="hdi_own_rank"))
    + geom_point(size=3)
    + geom_text(aes(label="text"), size=5)
    + labs(
        x="Rank HDI (2018)",
        y="Rank of custom HDI (2018)",
        title="Comparing two human development indices",
    )
)

**Figure 4.11 Scatterplot of ranks for HDI and alternative HDI index with text.**

You can see that in general the rankings are similar. If they were identical, the points in the scatterplot would form a straight upward-sloping line. They do not form a straight line, but there is a very strong positive correlation. There are, however, a few countries where the alternative definitions have caused a change in ranking, so let's look at where the alternative definitions have caused a big change in ranking using a difference.

In [ ]:
hdr_merged = hdr_merged.assign(
    rank_diff=hdr_merged[["hdi_own_rank", "hdi_rank_2018"]].diff(axis=1)[
        "hdi_rank_2018"
    ]
).sort_values("rank_diff", ascending=False)
hdr_merged.head(6)

In [ ]:
hdr_merged.tail(6)

From this, we can see that countries such as Djibouti, Sudan, Niger, and Burkina Faso do much better on the custom ranking. Meanwhile, Hungary, Czechia, and Georgia have fared somewhat less well on this new ranking.

This chapter used the following packages where *sys* is the Python version:

In [ ]:
%load_ext watermark
%watermark --iversions